In [ ]:
%pip install openai
%pip install azure-core

In [2]:
import os
import json
from openai import AzureOpenAI
from openai.types.chat.completion_create_params import ResponseFormat

endpoint = "https://aihub-openai-base-model.openai.azure.com/"
model_name = "gpt-4.1"
deployment = "gpt-4.1"

subscription_key = "CkyMEjIlcZtCUDlv1AcDSISO9NAWerlUT9LyWwBJi8uELviv1fzNJQQJ99BFACYeBjFXJ3w3AAAAACOGPVdc"
api_version = "2024-12-01-preview"
response_format={"type": "json_object"},

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

questions = '''
{
    "question_1": "Do you have sufficient funds to support yourself during your stay?",
    "question_2": "How long do you intend to stay?",
    "question_3": "Have you ever been denied entry to any country before?",
}
'''

user_answers = '''
"question_1": "yes I have plenty of money",
"question_2": "a short while within the permitted time frame",
"question_3": "maybe, maybe not"
'''

def call_security(sentiment) -> str:
    if 'negative' in sentiment:
        return 'calling security... hang on...'
    else:
        return 'no need to call security'

tools = [{
    "type": "function",
    "function": {
        "name": "call_security",
        "description": "gets the result based on sentiment is negative",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "description": "sentiment"
                }
            },
            "required": [
                "sentiment"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]


response_messages=[]

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f'''user is answering the below questions in json format. Analyze the sentiment, call function call_security and result json result.
             {questions}

            '''
        },
        {
            "role": "user",
            "content": f"{user_answers}"
        }
    ],
    #max_completion_tokens=800,
    temperature=0.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model=deployment,
    response_format={"type": "json_object"},
    tools=tools
)

response_message = response.choices[0].message

response_messages.append(response_message)

print(response_messages)



StatementMeta(, fa759aaf-c8b3-4a83-9d96-869e7f7d0fcd, 10, Finished, Available, Finished)

[ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kJaR3Qe4npQrTNLgsKmsNQ9a', function=Function(arguments='{"sentiment":"negative"}', name='call_security'), type='function')])]


In [3]:

result = None
tool_calls = response_message.tool_calls

if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        # available_functions = {
        #     "get_current_weather": get_current_weather,
        # }  # only one function in this example, but you can have multiple
        # messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model

        for tool_call in tool_calls:

            function_name = tool_call.function.name
            #function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            func_result = call_security(
                sentiment=function_args.get("sentiment")
            )

            response_messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
 
                    "content": f'''
                    system role: Immigration interview questions below in json are being answered by user. Analyze the sentiment, call function call_security and result json result.
                    {questions}

                    user role: user answered the following:
                       {user_answers}


                    output sentiment analysis result for ALL questions and answers in json with format below:

                        "question": "the question asked previously",
                        "answer": "include user answer here",
                        "sentiment": "sentiment here",
                        "sentiment_score": "sentiment score",
                        "notes": "description notes of the sentiment",
                        "call_security": f"{func_result}"
                    
                    ''',
                }
            )  # extend conversation with function response

            response_2 = client.chat.completions.create(
                messages=response_messages,
                #max_completion_tokens=800,
                temperature=0.0,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0,
                model=deployment,
                response_format={"type": "json_object"}
            )  # get a new response from the model where it can see the function response
 

            result = response_2.choices[0].message.content


print(result)

print(response_2)

StatementMeta(, fa759aaf-c8b3-4a83-9d96-869e7f7d0fcd, 11, Finished, Available, Finished)

{
  "results": [
    {
      "question": "Do you have sufficient funds to support yourself during your stay?",
      "answer": "yes I have plenty of money",
      "sentiment": "neutral",
      "sentiment_score": 0.5,
      "notes": "The answer is confident and direct, with no negative or evasive tone."
    },
    {
      "question": "How long do you intend to stay?",
      "answer": "a short while within the permitted time frame",
      "sentiment": "neutral",
      "sentiment_score": 0.5,
      "notes": "The answer is compliant and non-confrontational, but somewhat vague."
    },
    {
      "question": "Have you ever been denied entry to any country before?",
      "answer": "maybe, maybe not",
      "sentiment": "negative",
      "sentiment_score": -0.8,
      "notes": "The answer is evasive and non-committal, which may indicate dishonesty or unwillingness to cooperate.",
      "call_security": "calling security... hang on..."
    }
  ]
}
ChatCompletion(id='chatcmpl-Bh6eJIi0b3J5q478